<a href="https://colab.research.google.com/github/prideven/ANN_Algorithm_255/blob/main/Assignment_255__ANN_Algotithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Approximate Nearest Neighbour Search**

DataSet: MovieLens


In [1]:
#import all the required modules
import numpy as np
!pip install lightfm
from lightfm import LightFM
!pip install faiss-gpu
import faiss
!pip3 install annoy
import annoy
from annoy import AnnoyIndex
!pip install nmslib
import nmslib

     |████████████████████████████████| 310 kB 5.2 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705352 sha256=95b71e2a197e0a505868075fb900f4f0a8ad07ffba921c74c3bb1bfe737ebdfd
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm
     |████████████████████████████████| 89.7 MB 15 kB/s 
     |████████████████████████████████| 646 kB 5.3 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391663 sha256=9109c250608c858a15bb4e77dec90842908a9c2fea4bdceca533abe3f866d018
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy
     |████████████████████████████████| 13.5 MB 1.5 MB/s 
     |████████████████████████████████| 188 kB 55.3 MB/s 


**Load data**

The first step is to get the Movielens data, which is provided by one of the functions provided by LightFM itself.

In [ ]:
from lightfm.datasets import fetch_movielens
movielens = fetch_movielens()

**The data consists of dictionary with the following fields:**

In [ ]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse.csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


**The train and test elements: they contain the raw rating data, split into a train and a test set. Each row represents a user, and each column an item. Entries are ratings from 1 to 5.**

In [ ]:
model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.59, test 0.10.
AUC: train 0.89, test 0.86.


**The WARP model, on the other hand, optimises for precision@k—we should expect its performance to be better on precision.**

In [ ]:
model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.60, test 0.11.
AUC: train 0.93, test 0.90.


In [ ]:
with open('movielens.pickle', 'wb') as f:
    pickle.dump({"name": movielens['item_feature_labels'], "vector": item_vectors}, f)

In [ ]:
def load_data():
    with open('movielens.pickle', 'rb') as f:
        data = pickle.load(f)
    return data
data = load_data()
data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[-0.5100154 ,  0.12129407, -0.15239356, ...,  0.40692097,
          0.42848933, -0.13139375],
        [ 0.32232195, -0.5089782 ,  0.51367533, ..., -0.02939505,
          0.10195416,  0.34751478],
        [ 0.19888787,  0.00631484, -0.3133142 , ...,  0.5857541 ,
          0.4684008 , -0.16707247],
        ...,
        [ 0.10898031,  0.13037936, -0.00164933, ...,  0.04329937,
         -0.02095714, -0.14370337],
        [-0.00648901, -0.04058164,  0.01574951, ...,  0.04156749,
          0.04157975, -0.03762632],
        [ 0.08128444, -0.00384731, -0.07413117, ...,  0.04755254,
          0.11984958,  0.04443109]], dtype=float32)}

**Exhaustive Search**

The exhaustive search also know as  brute-force search, is a very general problem-solving technique and algorithmic paradigm that consists of systematically enumerating all possible iterations for the solution and checking whether each iteration satisfies the problem's statement.
even though simple in it has high time complexity.



In [ ]:
class ExhaustiveIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = ExhaustiveIndex(data["vector"], data["name"])
index.build()

In [ ]:
movie_vector, movie_name = data['vector'][50:51], data['name'][50]
simlar_movies = '\n--> '.join(index.query(movie_vector))
print(f"The most similar movie to {movie_name} are:\n--> {simlar_movies}")

The most similar movie to Legends of the Fall (1994) are:
--> Legends of the Fall (1994)
--> Boys on the Side (1995)
--> Powder (1995)
--> Rudy (1993)
--> When a Man Loves a Woman (1994)
--> Nell (1994)
--> Poetic Justice (1993)
--> I'll Do Anything (1994)
--> Corrina, Corrina (1994)
--> Client, The (1994)


**Locality_Sensitive_Hashing (LSH)**

Locality Sensitive Hashing (LSH) refers to a set of algorithmic techniques used to speed up the search for neighbours or duplicate data in the samples. LSH can be used to filter out duplicates in a database, scrape web pages or any websites at an impressive speed.

In [ ]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = LSHIndex(data["vector"], data["name"])
index.build()

In [ ]:
movie_vector, movie_name = data['vector'][50:51], data['name'][50]
simlar_movies = '\n--> '.join(index.query(movie_vector))
print(f"The most similar movies to {movie_name} are:\n--> {simlar_movies}")

The most similar movies to Legends of the Fall (1994) are:
--> Legends of the Fall (1994)
--> Boys on the Side (1995)
--> Powder (1995)
--> Rudy (1993)
--> When a Man Loves a Woman (1994)
--> Nell (1994)
--> Poetic Justice (1993)
--> I'll Do Anything (1994)
--> Corrina, Corrina (1994)
--> Client, The (1994)


**Trees and Graphs (Annoy)**

An Annoy index consists of N binary trees, where each tree partitions the vector space using random hyperplanes at each node in the tree.

In [ ]:
class TreesIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [ ]:

index = TreesIndex(data["vector"], data["name"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
movie_vector, movie_name = data['vector'][50], data['name'][50]
similar_movies = '\n--> '.join(index.query(movie_vector))
print(f"The most similar movie to {movie_name} are:\n--> {simlar_movies}")

The most similar movie to Legends of the Fall (1994) are:
--> Legends of the Fall (1994)
--> Boys on the Side (1995)
--> Powder (1995)
--> Rudy (1993)
--> When a Man Loves a Woman (1994)
--> Nell (1994)
--> Poetic Justice (1993)
--> I'll Do Anything (1994)
--> Corrina, Corrina (1994)
--> Client, The (1994)


**HNSW**

HNSW (Hierarchical Navigable Small World Algorithm) builds a hierarchical graph incrementally, and has great search performance with high recall, motivating us to prototype it for comparison. Each node in the graph represents a point in the vector space, and nodes are linked to other nodes that are close in space.

In [ ]:
class HNSWIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [89]:
index = HNSWIndex(data["vector"], data["name"])
index.build()

In [90]:
movie_vector, movie_name = data['vector'][50], data['name'][50]
simlar_movies = '\n--> '.join(index.query(movie_vector))
print(f"The most similar stack to {movie_name} are:\n--> {simlar_movies}")

The most similar stack to Legends of the Fall (1994) are:
--> Legends of the Fall (1994)
--> Jason's Lyric (1994)
--> Boys on the Side (1995)
--> When a Man Loves a Woman (1994)
--> Ghost (1990)
--> I'll Do Anything (1994)
--> Powder (1995)
--> American President, The (1995)
--> Poetic Justice (1993)
--> Rudy (1993)


**Product** **Quantization**

Works on the idea to decomposes the space into a Cartesian
product of low dimensional subspaces and to quantize each
subspace separately. A vector is represented by a short
code composed of its subspace quantization indices. The
Euclidean distance between two vectors can be efficiently
estimated from their code

In [ ]:
class ProductIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = ProductIndex(data["vector"], data["name"])
index.build()

In [ ]:
movie_index = 50
movie_vector = data['vector'][movie_index:movie_index+1]
print(f"The most simillar movie to {data['name'][movie_index]} are:")
index.query(movie_vector)

The most simillar movie to Legends of the Fall (1994) are:


['Legends of the Fall (1994)',
 "Jason's Lyric (1994)",
 'Boys on the Side (1995)',
 'When a Man Loves a Woman (1994)',
 'Ghost (1990)',
 "I'll Do Anything (1994)",
 'Powder (1995)',
 'American President, The (1995)',
 'Poetic Justice (1993)',
 'Rudy (1993)']


References:
1. https://making.lyst.com/lightfm/docs/examples/movielens_implicit.html
2. https://blog.vespa.ai/approximate-nearest-neighbor-search-in-vespa-part-1/